In [ ]:
from pyspark.sql.functions import col, current_timestamp,regexp_replace,when

In [ ]:
loan_schema="loan_id string,member_id string,loan_amount float,funded_amnt float,loan_term_months string,interest_rate float,monthly_installment float,issue_date string,loan_status string,loan_purpose string,loan_title string"

columns_to_check=["loan_amount","funded_amnt","loan_term_months","interest_rate","monthly_installment","issue_date","loan_status","loan_purpose"]

loan_purpose_lookup=["debt_consolidation","credit_card","home_improvement","other","major_purchase","medical","small_business","car","vacation","moving","house","wedding","renewable_energy","educational"]

In [ ]:
srv_date = dt.datetime.now().strftime('%Y-%m-%d')
print(srv_date)
loansPath = f"/user/{user_name}/lendingClub/srv/2024-01-07/loans/"
print(loansPath)
ln_csv_dest = f"/user/{user_name}/lendingClub/bv/csv/loans/"
ln_parq_dest = f"/user/{user_name}/lendingClub/bv/parquet/loans/"
print(ln_csv_dest)
print(ln_parq_dest)

In [ ]:
ln_DF = spark.read \
            .format("csv") \
            .schema(loan_schema) \
            .option("header",True) \
            .option("path",loansPath) \
            .load()

id_DF = ln_DF.withColumn("ingest_date", current_timestamp())

drop_DF = id_DF.na.drop(subset= columns_to_check)

In [ ]:
yr_DF = drop_DF.withColumn("loan_term_months", (regexp_replace("loan_term_months", " months", "").cast("float")/12).cast("int")) \
        .withColumnRenamed("loan_term_months","loan_term_years")

purp_DF = yr_DF.withColumn("loan_purpose", when(col("loan_purpose").isin(loan_purpose_lookup), col("loan_purpose")) \
                                            .otherwise("other"))

In [ ]:
loan_count = purp_DF.count()
addLog("ingested loans file with " + str(loan_count) + " records in BV")

purp_DF.write \
        .format("csv") \
        .option("header", True) \
        .option("path", ln_csv_dest) \
        .mode("overwrite") \
        .save()

purp_DF.write \
        .format("parquet") \
        .option("header", True) \
        .option("path", ln_parq_dest) \
        .mode("overwrite") \
        .save()

In [ ]:
loans_stat_bv = "Success"